In [1]:
import os
import glob
import cv2
import torch
import numpy as np
from transformers import DetrImageProcessor, DetrForObjectDetection
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torchvision.ops import box_iou


2024-08-18 03:13:23.318845: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 03:13:23.501571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 03:13:23.565740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 03:13:23.585559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 03:13:23.722504: I tensorflow/core/platform/cpu_feature_guar

In [10]:
keyframes_dir = '/media/daoan/T7 Shield2/VN_Multi_User_Video_Search/Keyframes'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
#     data_part = part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[part] =  dict()

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

In [13]:
class VisualEncoding:
    def __init__(self,
                 classes=('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
                          'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                          'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                          'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
                          'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
                          'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                          'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
                          'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
                          'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                          'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
                          'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
                          'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                          'scissors', 'teddy bear', 'hair drier', 'toothbrush'),
                 colors=('black', 'blue', 'brown', 'green', 'grey', 'orange_', 'pink', 'purple',
                         'red', 'white', 'yellow'),
                 row_str=["0", "1", "2", "3", "4", "5", "6"],
                 col_str=["a", "b", "c", "d", "e", "f", "g"]):

        self.classes = classes
        self.colors = colors
        self.classes2idx = dict()
        for i, class_ in enumerate(classes):
            self.classes2idx[class_] = i
        self.n_row = len(row_str)
        self.n_col = len(col_str)

        x_pts = np.linspace(0, 1, self.n_row + 1)
        y_pts = np.linspace(0, 1, self.n_col + 1)

        self.grid_bboxes = []
        self.grid_labels = []
        for i in range(self.n_row):
            for j in range(self.n_col):
                label = col_str[j] + row_str[i]
                self.grid_bboxes.append([x_pts[j], y_pts[i], x_pts[j + 1], y_pts[i + 1]])
                self.grid_labels.append(label)

        self.grid_bboxes = np.array(self.grid_bboxes)

    def visualize_grid(self, grid_vis=None):
        if grid_vis is None:
            grid_vis = np.zeros((500, 500, 1))

        vis_h, vis_w, _ = grid_vis.shape
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 2
        for i in range(self.n_row * self.n_col):
            x_start, y_start, x_end, y_end = self.grid_bboxes[i]
            label = self.grid_labels[i]
            org = (int((x_start + (x_end - x_start) / 2) * vis_w), int((y_start + (y_end - y_start) / 2) * vis_h))

            # Draw text
            grid_vis = cv2.putText(grid_vis, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
            # Draw grid
            grid_vis = cv2.rectangle(grid_vis, (int(x_start * vis_w), (int(y_start * vis_h))),
                                     (int(x_end * vis_w), int(y_end * vis_h)), color, thickness)
        plt.imshow(grid_vis)

    def encode_bboxes(self, bboxes, labels):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            labels: np.array: (n_bboxes, )
        '''
        iou = box_iou(torch.as_tensor(bboxes), torch.as_tensor(self.grid_bboxes))
        bboxes_idx, locs_idx = np.nonzero(iou.numpy())

        context = []
        for bbox_idx, loc_idx in zip(bboxes_idx, locs_idx):
            context.append(self.grid_labels[loc_idx] + self.classes[labels[bbox_idx]].replace(" ", ""))
        context = ' '.join(map(str, context))
        return context

    def encode_classes(self, labels):
        '''
        Args:
            labels: np.array: (n_bboxes, )
        '''
        unique_classes, counts = np.unique(labels, return_counts=True)
        context = []
        for unique_class, count in zip(unique_classes, counts):
            for i in range(count):
                context.append(self.classes[unique_class].replace(" ", "") + str(i))
        context = ' '.join(map(str, context))
        return context

    def encode_numbers(self, labels):
        '''
        Args:
            labels: np.array: (n_bboxes, )
        '''
        unique_classes, counts = np.unique(labels, return_counts=True)
        context = []
        for unique_class, count in zip(unique_classes, counts):
            context.append(self.classes[unique_class].replace(" ", "") + str(count))
        context = ' '.join(map(str, context))
        return context

    def encode_colors(self, bboxes, colors):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            colors: np.array: (n_bboxes, )
        '''
        iou = box_iou(torch.as_tensor(bboxes), torch.as_tensor(self.grid_bboxes))
        bboxes_idx, locs_idx = np.nonzero(iou.numpy())

        context = []
        for bbox_idx, loc_idx in zip(bboxes_idx, locs_idx):
            context.append(self.grid_labels[loc_idx] + colors[bbox_idx].replace("_", ""))
        context = ' '.join(map(str, context))
        return context

    def encode(self, bboxes=None, labels=None, bboxes_colors=None, colors=None):
        '''
        Args:
            bboxes: np.array: (n_bboxes, 4) - expected normalized bbox in form (x0, y0, x1, y1)
            labels: np.array: (n_bboxes, )
        '''
        results = dict()
        if bboxes is not None:
            results['bbox'] = self.encode_bboxes(bboxes, labels)
            results['class'] = self.encode_classes(labels)
        else:
            results['bbox'] = results['class'] = None

        if bboxes_colors is not None:
            results['color'] = self.encode_colors(bboxes_colors, colors)
        else:
            results['color'] = None

        return results

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(device)


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
import os
import torch
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as T

# Define the transform
transform = T.Compose([
    T.Resize((800, 800)),  # Resize to the required size (adjust as needed)
    T.ToTensor(),  # Convert to tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet means and stds (adjust if different for your model)
])

# Batch size and directories
bs = 4
save_dirs = ["bboxes_encoded", "classes_encoded", "number_encoded"]
encoder = VisualEncoding()
conf_threshold = 0.5

# Assuming 'device' is already defined, e.g., device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure the model is on the correct device
model.to(device)

# Create directories if they don't exist
for save_dir in save_dirs:
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    video_ids = sorted(video_keyframe_paths.keys())

    for save_dir in save_dirs:
        key_dir = os.path.join(save_dir, key)
        if not os.path.exists(key_dir):
            os.makedirs(key_dir)

    for video_id in tqdm(video_ids):
        video_keyframe_path = video_keyframe_paths[video_id]
        video_encoded_bboxes, video_encoded_classes, video_encoded_numbers = [], [], []

        for i in tqdm(range(0, len(video_keyframe_path), bs)):
            # Support batchsize inferencing
            image_paths = video_keyframe_path[i:i+bs]
            
            images = [Image.open(path).convert("RGB") for path in image_paths]
            images = [transform(image).unsqueeze(0).to(device) for image in images]  # Apply transform and move to device
            images = torch.cat(images)

            model.eval()
            with torch.no_grad():
                results = model(images)  # Perform inference
            
            # Check the structure of the result
            for result in results:
                # Print the result to understand its structure
                # Assuming the result is a dictionary with keys 'boxes', 'labels', and 'scores'
                if isinstance(result, dict):
                    bboxes = result.get('boxes').cpu().numpy().copy()
                    labels = result.get('labels').cpu().numpy().copy().astype(int)
                    scores = result.get('scores').cpu().numpy().copy()

                    # Filter out low-confidence results
                    high_conf_indices = scores >= conf_threshold
                    bboxes = bboxes[high_conf_indices]
                    labels = labels[high_conf_indices]

                    if len(bboxes) == 0:
                        video_encoded_bboxes.append("")
                        video_encoded_classes.append("")
                        video_encoded_numbers.append("")
                    else:
                        video_encoded_bboxes.append(encoder.encode_bboxes(bboxes, labels))
                        video_encoded_classes.append(encoder.encode_classes(labels))
                        video_encoded_numbers.append(encoder.encode_numbers(labels))
                else:
                    print("Unexpected result format:", type(result))

        # Save the video context txt
        with open(f"bboxes_encoded/{key}/{video_id}.txt", "w") as f:
            for item in video_encoded_bboxes:
                f.write("%s\n" % item)
        with open(f"classes_encoded/{key}/{video_id}.txt", "w") as f:
            for item in video_encoded_classes:
                f.write("%s\n" % item)
        with open(f"number_encoded/{key}/{video_id}.txt", "w") as f:
            for item in video_encoded_numbers:
                f.write("%s\n" % item)


  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/269 [00:00<?, ?it/s]

  1%|          | 2/269 [00:00<00:24, 10.79it/s]

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-15.0064,  -0.4377,  -4.1459,  ...,  -8.0918,  -4.4866,   9.6941],
         [-18.2308,  -2.3481,  -4.5492,  ...,  -8.6540,  -5.6152,  12.2046],
         [-18.0417,  -1.3560,  -6.1195,  ..., -10.9284,  -5.3899,  11.2066],
         ...,
         [-17.4687,  -1.2289,  -6.9598,  ...,  -6.3507,  -3.1151,   9.9956],
         [-18.3279,  -3.2072,  -6.6606,  ..., -11.3488,  -6.5307,  11.9128],
         [-18.0381,  -2.1562,  -4.3081,  ...,  -7.7382,  -5.4919,  12.0679]],

        [[-14.6960,  -0.0658,  -3.8947,  ...,  -7.7823,  -4.5178,   9.4281],
         [-18.2931,  -3.0905,  -4.3888,  ...,  -6.9935,  -5.8552,  12.0963],
         [-16.6944,  -0.7175,  -4.9582,  ...,  -9.1602,  -5.5355,  10.4543],
         ...,
         [-17.4453,  -1.1707,  -6.5351,  ...,  -5.9869,  -3.5557,   9.9831],
         [-18.5105,  -2.5208,  -5.9311,  ..., -10.3030,  -6.7576,  11.9855],
         [-16.9608,  -1.3454,  -5.9397,  ...,  -5.6322,  -3.729



  1%|▏         | 4/269 [00:00<00:23, 11.10it/s]

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-13.8887,   6.8728,  -4.9564,  ...,  -6.3692,  -4.8594,   6.2813],
         [-14.2749,   6.6047,  -5.5384,  ...,  -6.5083,  -5.0309,   7.0414],
         [-15.0092,   6.5997,  -3.9654,  ...,  -5.9606,  -6.9469,   6.3719],
         ...,
         [-14.6186,   1.0017,  -5.8676,  ...,  -6.5000,  -4.2980,   6.5992],
         [-15.5190,   5.6165,  -3.6941,  ...,  -8.1600,  -6.1118,   7.3561],
         [-12.4185,   1.8322,  -4.3913,  ...,  -4.9567,  -3.6637,   5.9353]],

        [[-13.8430,   6.8946,  -5.0161,  ...,  -6.6334,  -4.9593,   6.4429],
         [-13.4616,   7.2503,  -5.2002,  ...,  -6.9863,  -5.3047,   7.0820],
         [-14.6953,   3.5914,  -4.5250,  ...,  -7.0425,  -7.5981,   6.5418],
         ...,
         [-14.7325,   1.0669,  -6.5611,  ...,  -6.3324,  -4.6202,   6.8460],
         [-15.4680,   6.6763,  -4.7502,  ...,  -8.5353,  -6.5396,   7.5325],
         [-12.5980,   1.9118,  -4.9815,  ...,  -4.9394,  -4.036



  2%|▏         | 6/269 [00:00<00:25, 10.51it/s]

  3%|▎         | 8/269 [00:00<00:23, 10.88it/s]

...



 49%|████▊     | 131/269 [00:12<00:12, 11.27it/s]

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-17.4638,  -1.0881,  -3.0700,  ...,  -7.9647, -10.1231,   9.5234],
         [-16.2392,  -0.2931,  -6.7300,  ...,  -9.6692,  -2.6942,   9.1590],
         [-18.6434,  -1.0434,  -6.0524,  ..., -13.4796,  -8.6406,   9.7167],
         ...,
         [-17.9559,  -1.7678,  -5.0045,  ..., -12.6818,  -7.7291,   9.4566],
         [-12.8882,  -1.5969,  -4.0898,  ..., -13.1281,  -6.4122,   7.0405],
         [-16.6169,  -2.6600,  -4.5844,  ..., -11.7985,  -8.3207,   9.6943]],

        [[-17.6876,  -0.9991,  -3.3056,  ...,  -8.8557, -10.5438,   9.8114],
         [-16.7776,  -1.9824,  -6.9332,  ...,  -9.5567,  -4.1853,   9.5292],
         [-19.0434,  -1.5887,  -4.4868,  ..., -13.2195, -10.2094,  10.2293],
         ...,
         [-16.6852,   0.2609,  -4.9644,  ...,  -4.6892,  -6.0541,   9.0287],
         [-16.2625,  -2.5734,  -5.6306,  ..., -11.6514,  -7.3293,   9.1208],
         [-15.3747,  -1.6309,  -5.7092,  ..., -12.2600,  -5.777



 49%|████▉     | 133/269 [00:12<00:11, 11.52it/s]

  0%|          | 0/36 [00:12<?, ?it/s]

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-17.3799,  -0.6899,  -7.3343,  ...,  -9.0824,  -4.6747,  10.6513],
         [-18.2360,  -1.4384,  -8.2548,  ...,  -9.7290,  -4.3900,  11.1777],
         [-18.0677,   0.0465,  -8.5739,  ...,  -9.5549,  -5.1848,  11.1471],
         ...,
         [-17.1885,   5.1951,  -7.8788,  ...,  -7.9364,  -5.0623,  10.1157],
         [-15.2729,  11.7451,  -5.2781,  ...,  -9.9473,  -6.2565,   7.5780],
         [-17.6367,   4.8413,  -8.1154,  ...,  -8.3129,  -5.8120,  10.3899]],

        [[-18.5876,  -0.8709,  -7.2572,  ...,  -9.1576,  -5.5416,  10.7572],
         [-18.4655,  -0.9940,  -6.9411,  ...,  -9.6685,  -5.9337,  11.2879],
         [-18.8856,  -1.1597,  -5.3379,  ...,  -9.9241,  -6.4520,  11.2962],
         ...,
         [-17.2174,   5.8886,  -7.6019,  ...,  -7.8865,  -5.6073,   9.9750],
         [-15.5574,  11.1966,  -5.6650,  ...,  -9.7012,  -6.7694,   7.7707],
         [-19.3145,   0.3147,  -7.0568,  ...,  -9.9724,  -7.375

KeyboardInterrupt: 